<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-27 15:26:10
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.37 C
-------------------
Today PnL: -1.26 L (-0.91%)
Current PnL: -37.50 L (-23.44%)
CY Booked + Current PnL: -21.82 L (-13.64%)
-------------------
Total profit:  1.34 L
Total loss:  -38.84 L
-------------------
Total Booked + Current PnL: 5.16 L (3.88%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.45%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.04 C (76.04%)
Deployed:  1.33 C
Current:  1.37 C
CAGR/XIRR %: 1.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-0.74,5.82,11.20,17.67,17832.0,8750.0,159211.0,478.51,27.69,56.0,L-LC,11.04,185.0,0.49,1.18,27.81,XY25,ATH,MINING
50,MASFIN,-4.49,-0.89,22.66,21.57,22005.0,-870.0,97110.0,397.04,-14.63,47.0,H-SC,6.55,167.0,-0.04,0.72,41.71,XR,ATH,FINANCE
77,TTKPRESTIG,-1.09,-32.22,47.67,0.09,32563.0,-32467.0,68310.0,770.00,58.59,30.0,M-SC,11.62,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
51,MEDANTA,-1.44,-4.40,29.78,24.08,36144.0,-5580.0,121370.0,1486.00,-5.04,51.0,X-SC,7.96,87.0,-0.15,0.90,11.17,XY24,NTT,HEALTHCARE
37,ICICIGI,-2.16,0.88,18.30,19.34,36941.0,1757.0,201866.0,2252.93,-15.18,49.0,X-MC,1.09,66.0,0.05,1.50,16.96,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,4.46,-25.13,64.33,23.03,86247.0,-45002.0,134069.0,216.00,46.93,53.0,H-SC,5.50,166.0,-0.52,1.00,33.02,XR,NTT,BANKS
27,FINCABLES,4.16,15.22,80.45,107.91,136857.0,22470.0,170114.0,1641.55,3.50,74.0,M-SC,7.09,220.0,0.16,1.26,27.98,OX40N,ATH,CABLES
24,DIXON,4.09,-26.99,79.79,31.26,134430.0,-62288.0,168480.0,18931.72,-67.41,40.0,X-MC,24.11,54.0,-0.46,1.25,4.09,X40N,ATH,IT
65,SIEMENS,2.52,-8.09,36.56,25.51,62534.0,-15050.0,171045.0,4671.50,43.57,67.0,H-LC,3.56,51.0,-0.24,1.27,26.05,AR,ATH,ELECTRICAL
83,WHIRLPOOL,1.75,-26.35,147.01,81.92,124294.0,-30250.0,84548.0,2270.00,-56.86,58.0,M-SC,26.04,210.0,-0.24,0.63,18.76,XR,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SYMPHONY,-5.83,-39.74,65.88,-0.03,67946.0,-68005.0,103136.0,1306.0,-51.59,30.0,M-SC,9.31,205.0,-1.00,0.77,0.00,OX40N,NTT,DURABLES
5,ANGELONE,-5.43,-90.45,1201.16,24.20,349994.0,-276112.0,29138.0,3033.0,-86.38,34.0,X-SC,2.42,97.0,-0.79,0.22,22.03,X40N,NTT,FINANCE
55,RAJESHEXPO,-4.99,-72.29,261.10,0.07,99255.0,-99163.0,38014.0,518.0,1292.68,29.0,L-SC,9.98,268.0,-1.00,0.28,0.00,OX40N,NTT,JEWELLERY
44,IRCTC,-4.91,-31.82,124.30,52.93,229251.0,-86074.0,184434.0,1269.0,-49.51,30.0,H-MC,12.14,119.0,-0.38,1.37,0.00,XY24,BTT,TRAVEL
17,CAMS,-4.67,-11.49,40.64,24.48,90593.0,-28941.0,222915.0,950.0,-15.26,33.0,X-SC,5.71,89.0,-0.32,1.66,9.53,X40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-4.49,-0.89,22.66,21.57,22005.0,-870.0,97110.0,397.04,-14.63,47.0,H-SC,6.55,167.0,-0.04,0.72,41.71,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,4.16,15.22,80.45,107.91,136857.0,22470.0,170114.0,1641.55,3.50,74.0,M-SC,7.09,220.0,0.16,1.26,27.98,OX40N,ATH,CABLES
66,SIS,1.37,-29.71,72.52,21.26,56597.0,-32989.0,78043.0,528.00,1833.68,42.0,H-SC,5.52,168.0,-0.58,0.58,7.59,OX40N,NTT,MISC
77,TTKPRESTIG,-1.09,-32.22,47.67,0.09,32563.0,-32467.0,68310.0,770.00,58.59,30.0,M-SC,11.62,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
48,KANSAINER,1.02,-32.43,67.93,13.47,123778.0,-87453.0,182214.0,340.00,-41.86,37.0,H-SC,13.02,162.0,-0.71,1.35,2.43,XY24,NTT,PAINTS
41,INDIGOPNTS,-1.48,-33.69,50.83,0.02,58838.0,-58805.0,115754.0,1408.00,76.34,26.0,M-SC,12.21,222.0,-1.00,0.86,0.69,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1.02,22.05,18.96,45.2,44372.0,42288.0,234030.0,1856.0,39.7,65.0,X-MC,9.08,79.0,0.95,1.74,30.35,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-0.74,5.82,11.20,17.67,17832.0,8750.0,159211.0,478.51,27.69,56.0,L-LC,11.04,185.0,0.49,1.18,27.81,XY25,ATH,MINING
27,FINCABLES,4.16,15.22,80.45,107.91,136857.0,22470.0,170114.0,1641.55,3.50,74.0,M-SC,7.09,220.0,0.16,1.26,27.98,OX40N,ATH,CABLES
1,ABB,-0.94,11.47,30.59,45.57,89208.0,30005.0,291624.0,7934.00,-28.35,65.0,H-MC,8.80,125.0,0.34,2.17,29.50,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.94,11.47,30.59,45.57,89208.0,30005.0,291624.0,7934.00,-28.35,65.0,H-MC,8.80,125.0,0.34,2.17,29.50,AR,NTT,ELECTRICAL
27,FINCABLES,4.16,15.22,80.45,107.91,136857.0,22470.0,170114.0,1641.55,3.50,74.0,M-SC,7.09,220.0,0.16,1.26,27.98,OX40N,ATH,CABLES
50,MASFIN,-4.49,-0.89,22.66,21.57,22005.0,-870.0,97110.0,397.04,-14.63,47.0,H-SC,6.55,167.0,-0.04,0.72,41.71,XR,ATH,FINANCE
40,INDIAMART,-2.30,-8.44,125.78,106.72,142040.0,-10409.0,112927.0,4810.62,-58.79,39.0,H-SC,8.07,140.0,-0.07,0.84,12.97,AR,ATH,MISC
85,ZYDUSLIFE,-2.19,-3.20,40.11,35.63,81370.0,-6707.0,202867.0,1286.17,-15.77,54.0,H-MC,4.90,120.0,-0.08,1.51,12.94,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,0.10,-38.55,155.80,57.19,312234.0,-125720.0,200407.0,485.00,-64.88,19.0,X-SC,18.82,81.0,-0.40,1.49,0.10,XY24,NTT,FMCG
43,INFY,0.85,-11.50,51.69,34.24,143130.0,-35988.0,276900.0,1972.00,-30.33,24.0,X-LC,8.70,3.0,-0.25,2.06,1.92,X40,NTT,IT
53,PGHH,-1.08,-15.11,55.50,32.01,100602.0,-32256.0,181264.0,17616.87,-42.57,27.0,X-MC,9.63,57.0,-0.32,1.35,0.00,X40,ATH,FMCG
12,BATAINDIA,-0.77,-49.14,165.82,35.20,108523.0,-63224.0,65446.0,2096.00,-14.22,27.0,X-SC,22.33,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
34,HCLTECH,1.24,-9.76,37.23,23.83,81276.0,-23616.0,218308.0,1908.19,-5.29,30.0,X-LC,6.06,8.0,-0.29,1.62,4.74,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIGI,-2.16,0.88,18.30,19.34,36941.0,1757.0,201866.0,2252.93,-15.18,49.0,X-MC,1.09,66.0,0.05,1.50,16.96,X40,ATH,INSURANCE
78,UNITDSPR,-0.76,1.25,19.28,20.77,45707.0,2924.0,237068.0,1644.00,-10.17,47.0,X-MC,1.27,64.0,0.06,1.76,8.74,X40N,NTT,BREWERIES
22,DABUR,-1.31,1.20,42.00,43.70,105435.0,2968.0,251036.0,735.00,-8.20,53.0,X-MC,1.46,70.0,0.03,1.87,16.43,XY24,BTT,FMCG
59,RELIANCE,-0.95,-10.55,40.08,25.30,72049.0,-21201.0,179762.0,1952.00,-12.73,40.0,X-LC,2.31,21.0,-0.29,1.34,20.41,XY25,BTT,REFINERIES
5,ANGELONE,-5.43,-90.45,1201.16,24.20,349994.0,-276112.0,29138.0,3033.00,-86.38,34.0,X-SC,2.42,97.0,-0.79,0.22,22.03,X40N,NTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,-1.36,-33.13,145.49,64.15,231489.0,-78841.0,159110.0,3906.00,-49.40,32.0,X-SC,9.51,85.0,-0.34,1.18,0.0,XY24,BTT,CEMENT
12,BATAINDIA,-0.77,-49.14,165.82,35.20,108523.0,-63224.0,65446.0,2096.00,-14.22,27.0,X-SC,22.33,91.0,-0.58,0.49,0.0,X40,NTT,FOOTWEAR
58,RELAXO,-2.36,-55.22,240.08,52.31,156076.0,-80150.0,65010.0,1176.00,-52.28,33.0,X-SC,14.85,92.0,-0.51,0.48,0.0,X40N,NTT,FOOTWEAR
53,PGHH,-1.08,-15.11,55.50,32.01,100602.0,-32256.0,181264.0,17616.87,-42.57,27.0,X-MC,9.63,57.0,-0.32,1.35,0.0,X40,ATH,FMCG
52,PAGEIND,-0.51,-18.78,60.23,30.14,115967.0,-44514.0,192540.0,51418.64,-39.96,34.0,X-MC,18.42,55.0,-0.38,1.43,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-5.43,-90.45,1201.16,24.20,349994.0,-276112.0,29138.0,3033.0,-86.38,34.0,X-SC,2.42,97.0,-0.79,0.22,22.03,X40N,NTT,FINANCE
54,QUESS,0.70,-34.11,116.79,42.84,50023.0,-22174.0,42832.0,424.0,-57.02,39.0,X-SC,17.10,80.0,-0.44,0.32,3.03,XY24,NTT,MISC
58,RELAXO,-2.36,-55.22,240.08,52.31,156076.0,-80150.0,65010.0,1176.0,-52.28,33.0,X-SC,14.85,92.0,-0.51,0.48,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.77,-49.14,165.82,35.20,108523.0,-63224.0,65446.0,2096.0,-14.22,27.0,X-SC,22.33,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,-1.44,-4.40,29.78,24.08,36144.0,-5580.0,121370.0,1486.0,-5.04,51.0,X-SC,7.96,87.0,-0.15,0.90,11.17,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-1.51,-17.42,44.18,19.06,124100.0,-59270.0,280896.0,452.00,-54.71,39.0,X-LC,17.21,1.0,-0.48,2.09,1.71,X40,NTT,FMCG
73,TCS,-0.23,-24.29,63.23,23.58,207107.0,-105083.0,327546.0,4311.59,-34.45,31.0,X-LC,9.14,2.0,-0.51,2.44,2.63,X40,ATH,IT
43,INFY,0.85,-11.50,51.69,34.24,143130.0,-35988.0,276900.0,1972.00,-30.33,24.0,X-LC,8.70,3.0,-0.25,2.06,1.92,X40,NTT,IT
75,TMPV,-2.53,-15.19,57.21,33.33,158080.0,-49485.0,276315.0,600.00,-81.44,57.0,X-LC,3.90,4.0,-0.31,2.05,12.96,XY24,NTT,AUTO
34,HCLTECH,1.24,-9.76,37.23,23.83,81276.0,-23616.0,218308.0,1908.19,-5.29,30.0,X-LC,6.06,8.0,-0.29,1.62,4.74,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,-0.73,-29.30,87.97,32.89,48017.0,-22624.0,54583.0,1800.00,-357.00,64.0,L-MC,12.10,258.0,-0.47,0.41,50.32,XR,NTT,BANKS
67,SONACOMS,-2.01,-7.55,50.45,39.09,47181.0,-7639.0,93520.0,804.02,-26.68,59.0,M-MC,10.82,193.0,-0.16,0.70,32.30,AR,ATH,AUTO
19,COALINDIA,-0.74,5.82,11.20,17.67,17832.0,8750.0,159211.0,478.51,27.69,56.0,L-LC,11.04,185.0,0.49,1.18,27.81,XY25,ATH,MINING
7,ATULAUTO,-1.29,-14.40,70.06,45.57,118221.0,-28390.0,168742.0,844.00,3656.76,58.0,M-SC,6.68,248.0,-0.24,1.25,27.70,XY24,NTT,AUTO
27,FINCABLES,4.16,15.22,80.45,107.91,136857.0,22470.0,170114.0,1641.55,3.50,74.0,M-SC,7.09,220.0,0.16,1.26,27.98,OX40N,ATH,CABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,-0.69,-52.43,206.26,45.70,173494.0,-92701.0,84114.0,26.40,-12.72,61.0,M-SC,18.19,204.0,-0.53,0.63,38.59,XY24,NTT,TRAVEL
1,ABB,-0.94,11.47,30.59,45.57,89208.0,30005.0,291624.0,7934.00,-28.35,65.0,H-MC,8.80,125.0,0.34,2.17,29.50,AR,NTT,ELECTRICAL
27,FINCABLES,4.16,15.22,80.45,107.91,136857.0,22470.0,170114.0,1641.55,3.50,74.0,M-SC,7.09,220.0,0.16,1.26,27.98,OX40N,ATH,CABLES
7,ATULAUTO,-1.29,-14.40,70.06,45.57,118221.0,-28390.0,168742.0,844.00,3656.76,58.0,M-SC,6.68,248.0,-0.24,1.25,27.70,XY24,NTT,AUTO
11,BANDHANBNK,-2.12,-12.38,119.60,92.42,291921.0,-34478.0,244081.0,400.00,109.05,71.0,H-SC,8.10,174.0,-0.12,1.81,34.46,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.74
1,25,45.79
2,50,77.81


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.98
MC    30.80
LC    25.24
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.50
X40      24.24
X40N     12.94
AR        9.35
XY25      9.10
XR        8.71
OX40N     7.32
SR        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.21
H-SC    22.75
X-LC    21.75
M-SC    11.00
X-SC     9.52
H-MC     5.05
M-MC     2.13
H-LC     1.27
L-LC     1.18
M-LC     1.04
L-SC     0.71
L-MC     0.41
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.93,-15.27,54.26
IT,11.63,-40.21,110.48
FINANCE,11.22,-38.52,93.04
MISC,6.93,-39.45,96.49
ELECTRICAL,6.54,-6.71,45.65
PAINTS,5.14,-33.46,53.28
INSURANCE,4.97,-1.51,36.25
PHARMA,4.22,-3.49,36.07
AUTO,4.00,-15.88,60.06


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3426623.0,21
X40,1462761.0,15
X40N,1381715.0,10
XR,1319500.0,12
AR,1315220.0,10
OX40N,808546.0,10
XY25,393815.0,6
SR,303767.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3733668.0,24
X-SC,1523940.0,10
M-SC,1437798.0,14
X-MC,1391473.0,15
X-LC,1349201.0,12
H-MC,399829.0,3
L-SC,194604.0,2
M-LC,152404.0,1
M-MC,100647.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1261006.0      6
           AR         959889.0      5
           XR         844478.0      7
M-SC       XY24       838745.0      6
X-LC       X40        713341.0      6
X-SC       X40N       706171.0      4
           XY24       629890.0      4
X-MC       X40        561541.0      7
           X40N       441802.0      4
H-SC       OX40N      364528.0      4
M-SC       OX40N      344763.0      5
H-SC       SR         303767.0      2
X-LC       XY24       271590.0      2
           X40N       233742.0      2
H-MC       XY24       229251.0      1
X-MC       XY24       196141.0      2
           XY25       191989.0      2
X-SC       X40        187879.0      2
M-SC       XR         179252.0      2
H-MC       AR         170578.0      2
M-LC       XR         152404.0      1
X-LC       XY25       130528.0      2
L-SC       OX40N       99255.0      1
           XR          95349.0      1
M-SC       AR          75038.0      1
H-LC       AR          62534.0      1
M-MC       XY25        53466.0      1
L-MC       XR          48017.0      1
M-MC       AR          47181.0      1
L-LC       XY25        17832.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
